<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/twelvelabs-io/twelvelabs-developer-experience/blob/main/quickstarts/TwelveLabs_Quickstart_Generate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in  Colab</a>
  </td>
</table>

## Installation of TwelveLabs SDKs

In [ ]:
!pip install -U -q twelvelabs
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task

##**Configure your API key. Add API Key as a env variable.**
Signup for TwelveLabs and Get your API keys [here](https://playground.twelvelabs.xyz/dashboard/api-key). No credit card is required to use the Free plan. The Free plan includes indexing of 600 mins of videos, which is enough for a small project.

*To run the following cell in a*
* *Colab Notebook, your API key must be stored it in a Colab Secret named TL_API_KEY. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [this](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for an example.*
* *In other Python environments, You can also use Python variables using `os.environ()`*



In [2]:
from google.colab import userdata
TL_API_KEY=userdata.get('TL_API_KEY')

## **Step:1 - Create Index (You can also use an existing Index)**
Two main inputs needed for Index creation is the Model Type and Model Options. Learn more [here](https://docs.twelvelabs.io/docs/create-indexes).

**Twelve Labs provides two distinct model types** - embedding and generative, each serving unique purposes in multimodal video understanding.

* Embedding models (Marengo) : `marengo2.7` model is proficient at performing tasks such as search, enabling enhanced video understanding.
* Generative models (Pegasus): `pegasus1.2` model generates text based on your videos.
Learn more [here](https://docs.twelvelabs.io/docs/video-understanding-models)

> *Video Duration limits: For Marengo, it must be between 4 seconds and 2 hours (7,200s). For Pegasus, it must be between 4 seconds and 60 minutes (3600). In a future release, the maximum duration for Pegasus will be 2 hours (7,200 seconds).*


You can use both models on the same index but the limits of Pegasus will apply. E.g. If you create an Index with both `Marengo` and `Pegasus` then you will be limited to videos with maximum duration of 20mins.

**Model options** indicate the types of information a video understanding model will process. When creating a new index, you must specify the models and the associated model options that you want to enable.

The following model options are available:

* `visual`: Analyzes video content as you would see it, including actions, objects, events, text (OCR) and brand logos.
* `audio`: Analyzes video content as you would hear from it, including ambient sounds and human speech.

In [ ]:
import os
from glob import glob
# The path to the directory containing the videos you wish to upload.
VIDEO_PATH = "ostrich-racing-every-tuesday-thursday.mp4"  # In a colab Notebook, just upload the file in the root folder and just give the filename.

client = TwelveLabs(api_key=TL_API_KEY)

index = client.index.create(
    name="Temp-Generate-2",
    models=[
        {
            "name": "pegasus1.2",
            "options": ["visual", "audio"],
        }
    ],
    addons=["thumbnail"], # Optional
)
print(f"Created index: id={index.id} name={index.name} models={index.models}")


## **Step:2 - Upload videos to an Index**

The platform offers the following options for uploading single videos:

* Upload from the local file system: Use this option to upload a single video file from the local file system.

* Upload from a direct URL: Use this option to upload a video file from a publicly accessible URL. The platform will retrieve the file directly from the specified URL, so your application doesn't have to store the video locally and upload it.

The video you wish to upload must meet the following requirements:
* Video resolution: Must be greater or equal than 360p and less or equal than 4K. For consistent search results, Twelve Labs recommends you upload 360p videos.
* Video and audio formats: The video files you wish to upload must be encoded in the video and audio formats listed on the FFmpeg Formats Documentation page. For videos in other formats, contact us at support@twelvelabs.io.
* Duration: For Marengo, it must be between 4 seconds and 2 hours (7,200s). For Pegasus, it must be between 4 seconds and 20 minutes (1200s). In a future release, the maximum duration for Pegasus will be 2 hours (7,200 seconds).
* File size: Must not exceed 2 GB.
* Audio track: If the `audio` model option is selected, the video you're uploading must contain an audio track.

Learn more in [docs](https://docs.twelvelabs.io/docs/upload-single-videos)

In [19]:
video_files = glob(VIDEO_PATH)
if(len(video_files)==0):
  raise Exception("Video file was not provided")
for video_file in video_files:
  print(f"Uploading {video_file}")
  task = client.task.create(index_id=index.id, file=video_file, language="en")
  print(f"Created task: id={task.id}")

  # (Optional) Monitor the video indexing process
  # Utility function to print the status of a video indexing task
  def on_task_update(task: Task):
          print(f"  Status={task.status}")
  task.wait_for_done(sleep_interval=50, callback=on_task_update)
  if task.status != "ready":
      raise RuntimeError(f"Indexing failed with status {task.status}")
  print(f"Uploaded {video_file}. The unique identifer of your video is {task.video_id}.")


Uploading ostrich-racing-every-tuesday-thursday.mp4
Created task: id=6660e72ce6fb7df29de0e5a0
  Status=pending
  Status=pending
  Status=indexing
  Status=ready
Uploaded ostrich-racing-every-tuesday-thursday.mp4. The unique identifer of your video is 6660e72cd22b3a3c97bf134d.


## **Step:3 - Generate Summaries and Chapters**

In [ ]:
#Get all videos in an Index
videos = client.index.video.list(index.id)
for video in videos:
    print(f"Generating text for {video.id}")

    res = client.generate.summarize(video_id=video.id, type="summary")
    print(f"Summary: {res.summary}")

    print("Chapters:")
    res = client.generate.summarize(video_id=video.id, type="chapter")
    for chapter in res.chapters:
        print(
            f"  chapter_number={chapter.chapter_number} chapter_title={chapter.chapter_title} chapter_summary={chapter.chapter_summary} start={chapter.start} end={chapter.end}"
        )

    print("Highlights:")
    res = client.generate.summarize(video_id=video.id, type="highlight")
    for highlight in res.highlights:
        print(
            f"  Highlight={highlight.highlight} start={highlight.start} end={highlight.end}"
        )

    res = client.generate.text(video_id=video.id, prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization).")
    print(f"Open-ended Text: {res.data}")

Generating text for 6660e72cd22b3a3c97bf134d
Title: Ostrich Racing in the Marketplace: Every Tuesday and Thursday
Topics=[' Ostrich racing in a marketplace setting, mentioned every Tuesday and Thursday.']
Hashtags=['perioddrama', 'marketplace', 'traditionalattire', 'ostrichracing', 'outdoorsetting', 'historicalscene', 'communalarea']
Summary: The video presents a captivating scene set in what appears to be a bustling marketplace from a historical or period drama. At the heart of this scene are two main characters, engaging in a friendly and amicable conversation amidst the lively atmosphere of the marketplace. The first character, a man adorned with a turban and traditional attire, exudes a sense of warmth and camaraderie as he talks, his face breaking into a smile. His counterpart, another man characterized by a mustache and beard, dons a white shirt with rolled-up sleeves, signaling a more casual demeanor. He listens attentively to the first man, his own expression mirroring the posi